In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import matplotlib.pyplot as plt


In [3]:
work_path = '/content/gdrive/Shared drives/algo308/marketdata/stock/'
os.chdir(work_path)

### Define Parameters

In [4]:
STOP_PROFIT = 1
STOP_LOSS = 2
TIME_OUT = 3

In [5]:
def numtotime(num):
    ms = num % 1000
    num //= 1000
    s = num % 100
    num //= 100
    mi = num % 100
    num //= 100
    h = num % 100
    num = num // 100
    # print(h, mi, s, ms)
    return dt.datetime(2020, 9, 21, h, mi, s, ms) # TODO

# Hft_strategy

In [6]:
class Hft_strategy:
    def __init__(self, lob_file, tick_file):
        self.lob_file = open(lob_file)
        self.tick_file = open(tick_file)
        # self.df = pd.read_csv(tick_file)
        # self.df["time"] = self.df["time"].apply(func = numtotime)
        # self.df.set_index("time", inplace = True)
        self.lob_col = self.lob_file.readline()[:-1].split(",")
        self.tick_col = self.tick_file.readline()[:-1].split(",")
        self.ref_num = 0
        self.num_buy = 0
        self.num_sell = 0
        self.ask1_p = 0
        self.ask2_p = 0
        self.ask3_p = 0
        self.ask4_p = 0
        self.ask5_p = 0
        self.ask1_q = 0
        self.ask2_q = 0
        self.ask3_q = 0
        self.ask4_q = 0
        self.ask5_q = 0
        self.bid1_p = 0
        self.bid2_p = 0
        self.bid3_p = 0
        self.bid4_p = 0
        self.bid5_p = 0
        self.bid1_q = 0
        self.bid2_q = 0
        self.bid3_q = 0
        self.bid4_q = 0
        self.bid5_q = 0
        self.daily_high = 0
        self.daily_low = 10000

        self.stop_profit = 0
        self.stop_loss = 0

        self.profit = 0
        self.cost = 0

        self.num_win = 0
        self.num_loss = 0

        self.order_time = 0
        self.stop_time = None

        self.position = []
        self.current_time = 0

        self.total_cost = 0

    def get_result(self):
        return self.profit, self.total_cost, self.num_win, self.num_loss

    def lob_update(self, lob_row):
        pass

    def tick_update(self, tick_row):
        pass

    def could_buy(self):
        pass

    def buy(self):
        pass

    def could_sell(self):
        pass

    def sell(self):
        pass

    def could_order(self):
        pass
        
    def order(self):
        pass

    def task(self):
        pass
    def show_result(self):
        pass

# YY13
### 多了偵測賣壓
#### self.bid1_p > self.weighted_p * self.theta
#### self.num_sell > self.num_buy * self.beta
#### 不同的position list length


In [7]:
class YY13(Hft_strategy):
    def __init__(self, lob_file, tick_file, beta, time_out, gamma, cool_down_min, stop_loss_factor, theta, position_max_q):
        super().__init__(lob_file, tick_file)
        self.tick_list = []
        self.beta = beta
        self.time_out = time_out
        self.order_p = np.array([])
        self.gamma = gamma
        self.in_history = []
        self.out_history = []
        self.in_time = []
        self.out_time = []
        self.cool_down_min = cool_down_min
        self.stop_loss_factor = stop_loss_factor
        self.theta = theta
        self.weighted_p = 0
        self.total_trade_q = 0
        self.total_trade_p = 0
        self.position_max_q = position_max_q
        self.position = np.array([])
        self.weighted_p_list = []
        self.weighted_p_time_list = []
        self.sell_buy_ratio_list = []


    def task(self):
        temp_lob = self.lob_file.readline()[:-1].split(",")
        temp_tick = self.tick_file.readline()[:-1].split(",")
        lob_row = pd.Series(data = temp_lob, index=self.lob_col)
        tick_row = pd.Series(data = temp_tick, index = self.tick_col)
        self.lob_datatype_conversion(lob_row)
        self.tick_datatype_conversion(tick_row)
        next_lob_time = lob_row['time']
        next_tick_time = tick_row['time']
        self.current_time = min(next_lob_time, next_tick_time)

        c = 0
        flag = 1
        while(self.current_time <= 130000000):
            c += 1                        
            if (next_lob_time < next_tick_time):
                self.lob_update(lob_row)
                temp_lob = self.lob_file.readline()[:-1].split(",")
                lob_row = pd.Series(data = temp_lob, index=self.lob_col)
                self.lob_datatype_conversion(lob_row)
                next_lob_time = lob_row['time']
            else:
                self.tick_update(tick_row)
                temp_tick = self.tick_file.readline()[:-1].split(",")
                tick_row = pd.Series(data = temp_tick, index = self.tick_col)
                self.tick_datatype_conversion(tick_row)
                next_tick_time = tick_row['time']
            self.current_time = min(next_lob_time, next_tick_time)
            if self.could_sell():
                self.sell()
            self.buy(tick_row)
            '''
            buy_type = self.could_buy(tick_row)
            if buy_type == STOP_PROFIT:
                self.buy()
                self.out_time.append(self.current_time)
                self.out_history.append(self.order_p)
                self.profit += self.cost - self.order_p 
            elif buy_type == STOP_LOSS or buy_type == TIME_OUT:
                self.buy()
                self.out_time.append(self.current_time)
                self.out_history.append(self.ask1_p)
                self.profit += -self.ask1_p + self.cost
                print("-"*30)
            '''
        self.clear_position(tick_row)

    
    def lob_datatype_conversion(self, lob_row):
        lob_row['time'] = int(lob_row['time'])
        lob_row['ask1_price'] = float(lob_row['ask1_price'])
        lob_row['ask2_price'] = float(lob_row['ask2_price'])
        lob_row['ask3_price'] = float(lob_row['ask3_price'])
        lob_row['ask4_price'] = float(lob_row['ask4_price'])
        lob_row['ask5_price'] = float(lob_row['ask5_price'])
        lob_row['bid1_price'] = float(lob_row['bid1_price'])
        lob_row['bid2_price'] = float(lob_row['bid2_price'])
        lob_row['bid3_price'] = float(lob_row['bid3_price'])
        lob_row['bid4_price'] = float(lob_row['bid4_price'])
        lob_row['bid5_price'] = float(lob_row['bid5_price'])
        lob_row['ask1_qty'] = int(float(lob_row['ask1_qty']))
        lob_row['ask2_qty'] = int(float(lob_row['ask2_qty']))
        lob_row['ask3_qty'] = int(float(lob_row['ask3_qty']))
        lob_row['ask4_qty'] = int(float(lob_row['ask4_qty']))
        lob_row['ask5_qty'] = int(float(lob_row['ask5_qty']))
        lob_row['bid1_qty'] = int(float(lob_row['bid1_qty']))
        lob_row['bid2_qty'] = int(float(lob_row['bid2_qty']))
        lob_row['bid3_qty'] = int(float(lob_row['bid3_qty']))
        lob_row['bid4_qty'] = int(float(lob_row['bid4_qty']))
        lob_row['bid5_qty'] = int(float(lob_row['bid5_qty']))

    def tick_datatype_conversion(self, tick_row):
        tick_row['time'] = int(tick_row['time'])
        tick_row['price'] = float(tick_row['price'])
        tick_row['qty'] = int(tick_row['qty'])

    def lob_update(self, lob_row):
        self.ask1_p = lob_row['ask1_price'] 
        self.ask2_p = lob_row['ask2_price']
        self.ask3_p = lob_row['ask3_price']
        self.ask4_p = lob_row['ask4_price']
        self.ask5_p = lob_row['ask5_price']
        self.ask1_q = lob_row['ask1_qty']
        self.ask2_q = lob_row['ask2_qty']
        self.ask3_q = lob_row['ask3_qty']
        self.ask4_q = lob_row['ask4_qty']
        self.ask5_q = lob_row['ask5_qty']
        self.bid1_p = lob_row['bid1_price']
        self.bid2_p = lob_row['bid2_price']
        self.bid3_p = lob_row['bid3_price']
        self.bid4_p = lob_row['bid4_price']
        self.bid5_p = lob_row['bid5_price']
        self.bid1_q = lob_row['bid1_qty']
        self.bid2_q = lob_row['bid2_qty']
        self.bid3_q = lob_row['bid3_qty']
        self.bid4_q = lob_row['bid4_qty']
        self.bid5_q = lob_row['bid5_qty']

    def tick_update(self, tick_row):
        self.daily_low = min(self.daily_low, tick_row['price'])
        self.daily_high = max(self.daily_high, tick_row['price'])
        # if self.current_time > 90100000:
            # self.total_trade_q += tick_row['qty'] 
            # self.total_trade_p += tick_row['price'] * tick_row['qty'] 
            # self.weighted_p = self.total_trade_p / self.total_trade_q

        self.total_trade_p += tick_row['price'] * tick_row['qty'] 
        self.total_trade_q += tick_row['qty'] 

        if (tick_row['price'] >= self.ask1_p):
            tick_row['buy_or_sell'] = "buy"
            self.num_buy += tick_row['qty'] 
        elif (tick_row['price'] <= self.bid1_p):
            tick_row['buy_or_sell'] = "sell"
            self.num_sell += tick_row['qty']
        else:
            tick_row['buy_or_sell'] = "QAO"
            
        if len(self.tick_list) < 100:
            self.tick_list.append(tick_row)
        else:
            self.total_trade_p -= self.tick_list[0]['price'] * self.tick_list[0]['qty'] 
            self.total_trade_q -= self.tick_list[0]['qty']
            if self.tick_list[0]['buy_or_sell'] == "buy":
                self.num_buy -= self.tick_list[0]['qty']
            elif self.tick_list[0]['buy_or_sell'] == "sell":
                self.num_sell -= self.tick_list[0]['qty']
            else:
                pass
            self.tick_list.pop(0)
            self.tick_list.append(tick_row)
            
    def could_sell(self):
        if self.bid1_p > self.weighted_p * self.theta \
            and self.bid1_p < self.daily_high * self.gamma\
            and self.current_time >= self.order_time + self.cool_down_min \
            and len(self.position) <= self.position_max_q \
            and self.current_time > 90200000\
            and self.num_sell > self.num_buy * self.beta\
            and self.bid2_q:
            return True
        else:
            return False

    def sell(self):
        self.in_history.append(self.bid1_p)
        self.in_time.append(self.current_time)
        self.position = np.append(self.position, self.bid1_p)
        self.order_time = self.current_time
        self.cost = self.bid1_p
        self.order_p = np.append(self.order_p, self.bid2_p)

        self.total_cost += self.cost

    def could_buy(self, tick_row):
        # stop profit
        if len(self.position) and tick_row['price'] <= np.max(self.order_p):
            return STOP_PROFIT
        elif len(self.position) and (tick_row['price'] >= self.position * self.stop_loss_factor).any():
            return STOP_LOSS
        # # time out
        # elif len(self.position) and self.current_time - self.order_time >= self.time_out:
        #    print("time out")
        #    return False
        else:
            return False
        # keep position

        # no position

    def buy(self, tick_row):
        res = self.could_buy(tick_row)
        if res == STOP_PROFIT:
            self.out_history.append(tick_row['price'])
            self.out_time.append(self.current_time)
            buy_index = np.argwhere(self.order_p >= tick_row['price'])
            self.profit += (self.position-self.order_p)[buy_index].sum()
            self.position = np.delete(self.position, buy_index)
            self.order_p = np.delete(self.order_p, buy_index)
        elif res == STOP_LOSS:
            self.out_history.append(tick_row['price'])
            self.out_time.append(self.current_time)
            buy_index = np.argwhere(tick_row['price'] >= self.position * self.stop_loss_factor)
            self.profit += (self.position-tick_row['price'])[buy_index].sum()
            self.position = np.delete(self.position, buy_index)
            self.order_p = np.delete(self.order_p, buy_index)

        # self.order_time = self.current_time
        
    def clear_position(self, tick_row):
        self.profit += np.sum(self.position-tick_row["price"])
            

    def show_result(self):
        
        
        if len(self.in_time):
        
            plt.figure(figsize=(20,10))
            plt.subplot(311)
            plt.title("price")
            plt.plot(self.df.index, self.df.loc[:, 'price'], label = 'price')
            plt.plot([numtotime(x) for x in self.weighted_p_time_list], self.weighted_p_list, label = 'weighted_p')
            plt.scatter([numtotime(x) for x in self.in_time], self.in_history, marker='v', s= 144, c="red")
            plt.scatter([numtotime(x) for x in self.out_time], self.out_history, marker='^', s = 144, c="lime")
            plt.legend()
        else:
            plt.figure(figsize=(20,10))
            plt.subplot(311)
            plt.title("price")
            plt.plot(self.df.index, self.df.loc[:, 'price'], label = 'price')
            plt.plot([numtotime(x) for x in self.weighted_p_time_list], self.weighted_p_list, label = 'weighted_p')
            plt.legend()
        plt.subplot(312)
        #plt.figure(figsize=(20,10))
        plt.title("qty")
        plt.plot(self.df.index, self.df.loc[:, "qty"])
        plt.subplot(313)
        plt.title("sell buy ratio")
        plt.yscale("log")
        self.weighted_p_time_list.append(133000000)
        self.sell_buy_ratio_list.append(self.sell_buy_ratio_list[-1])
        plt.plot([numtotime(x) for x in self.weighted_p_time_list], self.sell_buy_ratio_list, label = "sell")
        plt.plot([numtotime(x) for x in self.weighted_p_time_list], [1/x if x != 0 else 20 for x in self.sell_buy_ratio_list], label = 'buy')
        plt.plot([numtotime(self.weighted_p_time_list[0]), numtotime(self.weighted_p_time_list[-1])], [self.beta, self.beta])
        plt.legend()

        
        plt.show()
        


#### back_test

In [8]:
import datetime as dt


def back_test(start_date, end_date, sid, beta, time_out, gamma, cool_down_min, stop_loss_factor, theta, position_max_q):
    
    cur_day = None
    result_list = []
    for file in sorted(os.listdir()):
        if "zip" not in file:
            # print(file)
            year, month, date = [int(x) for x in file.split("-")]

            cur_day = dt.datetime(year, month, date)
            if end_date >= cur_day >= start_date:
                sid_type = "otc"
                lob_path = os.path.join(work_path, file, sid, sid_type + "_lob_" + sid + ".csv")
                tick_path = os.path.join(work_path, file, sid, sid_type + "_tick_" + sid + ".csv")

                if not os.path.exists(lob_path):
                    sid_type = "tse"
                    lob_path = os.path.join(work_path, file, sid, sid_type + "_lob_" + sid + ".csv")
                    tick_path = os.path.join(work_path, file, sid, sid_type + "_tick_" + sid + ".csv")
                temp = YY13(lob_path, tick_path, beta, time_out, gamma, cool_down_min, stop_loss_factor, theta, position_max_q)
                temp.task()
                result_list.append(temp.get_result())
                with open("../result_YY13_1004.csv", "a") as f:
                    f.write("\n")
                    f.write(",".join([str(x) for x in ["YY13", cur_day, sid, beta, time_out, gamma, \
                                                        cool_down_min, stop_loss_factor, theta, position_max_q, temp.profit, temp.total_cost]]))
                    
    return result_list 

def calculate_profit(res_list):
    s = 0
    c = 0
    for x in res_list:
        s+=x[0]
        c+=x[1]
    return s, c


#### RUN

# Grid Search

In [9]:
from tqdm import tqdm
STOCK_LIST = ["6462", "6278", "6488", "3037", "2330", "3406", "2317", "6505", "2412", "1301", "1326", "2882", "1303", "2881", "3008", "1216", "2891", "2002", "3045", "2454", "2886", "2912", "2308", "3711", "2892"]

SEC = 1000
MIN = 100000
COOL_DOWN_MIN_LIST = [1*MIN]
THETA_LIST = [1] 
BETA_LIST = [2]
# position_max_q = [1, 2]
POSITION_MAX_Q_LIST = [5, 10, 15]
for sid in tqdm(STOCK_LIST[-1]):
    print()
    print(sid)
    for position_max_q in POSITION_MAX_Q_LIST[-1]:
        for cool in COOL_DOWN_MIN_LIST[-1]:
            for beta in BETA_LIST[-1]:
                para = {"sid": sid, "beta":beta, "time_out": 2000000, "gamma": 1, "cool_down_min": cool, "stop_loss_factor": 1.02, "theta": 1, "position_max_q": position_max_q}
                res = back_test(dt.datetime(2020, 7, 1), dt.datetime(2020, 9, 25), **para)
                profit, cost = calculate_profit(res)
                print(para)
                print("profit:", profit)
                print("cost:", cost)
                print()

  0%|          | 0/4 [00:00<?, ?it/s]


2


TypeError: ignored